### 병원 네이버 플레이스 방문자 리뷰 크롤링
- https://pcmap.place.naver.com/hospital/1623603444/review/visitor?entry=bmp&from=map&fromPanelNum=2&timestamp=202309081302&x=126.89195542957395&y=37.50627134485392

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import pymongo as mg

In [2]:
client = mg.MongoClient(host='mongodb://localhost:27017')
database = client['ns_os_reviews']
collection = database['naver_place']

In [3]:
# open chrome browser
browser = webdriver.Chrome(executable_path='../../../../chromedriver.exe')

In [4]:
hospital_data = [
    # 국내 척추 병원 리스트(검색 통해 선정)
    ["서울바른병원", 'https://pcmap.place.naver.com/hospital/37427012/home?entry=bmp&from=map&fromPanelNum=2&timestamp=202309111054&x=126.92009780794396&y=37.500217907168434'],
    ["바로척척의원", 'https://pcmap.place.naver.com/hospital/37902519/home?entry=bmp&from=map&fromPanelNum=2&x=126.8395816&y=37.5588518&timestamp=202309111104'],
    ["연세더바로신경외과정형외과의원", 'https://pcmap.place.naver.com/hospital/1290271026/home?entry=bmp&from=map&fromPanelNum=2&timestamp=202309111106&x=127.02811010000002&y=37.525109099999995'],
    ["참포도나무병원", 'https://pcmap.place.naver.com/hospital/30805681/home?entry=bmp&from=map&fromPanelNum=2&timestamp=202309111107&x=127.0386971&y=37.479931300000004'],
    ["서울병원", 'https://pcmap.place.naver.com/hospital/21560637/home?entry=bmp&from=map&fromPanelNum=2&x=127.12467150000002&y=37.5039463&timestamp=202309111121'],
    ["바로척척의원", 'https://pcmap.place.naver.com/hospital/37281381/home?entry=bmp&from=map&fromPanelNum=2&x=127.1341414&y=37.5357686&timestamp=202309111127'],
    ["의료법인본플러스재단분당병원", 'https://pcmap.place.naver.com/hospital/20742703/home?entry=bmp&from=map&fromPanelNum=2&x=127.1064897&y=37.3636242&timestamp=202309111137'],
    ["기대플러스병원", 'https://pcmap.place.naver.com/hospital/86958121/home?entry=bmp&from=map&fromPanelNum=2&x=126.7364401&y=37.7287226&timestamp=202309111138'],
    ["서울바른세상병원", 'https://pcmap.place.naver.com/hospital/37445922/home?entry=bmp&from=map&fromPanelNum=2&timestamp=202309111139&x=126.89781340040102&y=37.47119169978151'],
    ["연세백퍼센트병원", 'https://pcmap.place.naver.com/hospital/1247579806/home?entry=bmp&from=map&fromPanelNum=2&timestamp=202309111140&x=126.7258778&y=37.4944746'],
    ["오케이병원", 'https://pcmap.place.naver.com/hospital/35913572/home?entry=bmp&from=map&fromPanelNum=2&x=126.73374058126338&y=37.52834095985524&timestamp=202309111141'],
    ["열린연세정형외과의원", 'https://pcmap.place.naver.com/hospital/1082253030/home?entry=bmp&from=map&fromPanelNum=2&timestamp=202309111141&x=126.93011169050402&y=37.36027717232209'],
    ["나사렛국제병원", 'https://pcmap.place.naver.com/hospital/13349355/home?entry=bmp&from=map&fromPanelNum=2&timestamp=202309111147&x=126.67051078074321&y=37.40808057937737'],
    ["나누리병원 주안", 'https://pcmap.place.naver.com/hospital/37350818/home?entry=bmp&from=map&fromPanelNum=2&x=126.69068909963093&y=37.45093300081088&timestamp=202309111150'],
    ["강북연세병원", 'https://pcmap.place.naver.com/hospital/13098049/home?entry=bmp&from=map&fromPanelNum=2&timestamp=202309111155&x=127.07550329951835&y=37.61918910116395'],

    # 보건복지부 제4기 1,2,3차년도 전문병원 지정기관
    # http://www.mohw.go.kr/react/modules/linkDownload.jsp?filePath=/upload/content_data/2021/0611.hwp&fileName=%EC%A0%9C4%EA%B8%B0%EC%A0%84%EB%AC%B8%EB%B3%91%EC%9B%90%EC%A7%80%EC%A0%95%EA%B8%B0%EA%B4%80.hwp
    ["CM병원", 'https://pcmap.place.naver.com/hospital/11559137/home?entry=pll&from=map&fromNxList=true&fromPanelNum=2&timestamp=202309151618&x=126.90418620000102&y=37.5188597000034'],
    ["서울부민병원", 'https://pcmap.place.naver.com/hospital/20149916/home?entry=pll&from=map&fromNxList=true&fromPanelNum=2&timestamp=202309151621&x=126.850950000384&y=37.5568917009414'],
    ["바로선병원", 'https://pcmap.place.naver.com/hospital/13009162/home?entry=bmp&from=map&fromPanelNum=2&x=127.04213750000216&y=37.660373699086705&timestamp=202309151623'],
    ["연세사랑병원", 'https://pcmap.place.naver.com/hospital/34358789/home?entry=bmp&from=map&fromPanelNum=2&x=126.9937383&y=37.4740477&timestamp=202309151625'],
    ["부민병원", 'https://pcmap.place.naver.com/hospital/11559118/home?entry=bmp&from=map&fromPanelNum=2&x=129.011254993713&y=35.2119078373885&timestamp=202309151626'],
    ["해운대부민병원", 'https://pcmap.place.naver.com/hospital/37021161/home?entry=bmp&from=map&fromPanelNum=2&x=129.15567528978605&y=35.16144983422887&timestamp=202309151626'],
    ["부산고려병원", 'https://pcmap.place.naver.com/hospital/12320216/home?entry=bmp&from=map&fromPanelNum=2&timestamp=202309151628&x=129.09262710000965&y=35.134914899995984'],
    ["부산본병원", 'https://pcmap.place.naver.com/hospital/13058320/home?entry=bmp&from=map&fromPanelNum=2&timestamp=202309151628&x=128.9698369191791&y=35.1062486721597'],
    ["의료법인 센텀의료재단 부산센텀병원", 'https://pcmap.place.naver.com/hospital/12407215/home?entry=bmp&from=map&fromPanelNum=2&x=129.1145750493048&y=35.16636282904132&timestamp=202309151629'],
    ["이춘택병원", 'https://pcmap.place.naver.com/hospital/11729486/home?entry=bmp&from=map&fromPanelNum=2&timestamp=202309151630&x=127.0157405&y=37.2718951'],
    ["바른세상병원", 'https://pcmap.place.naver.com/hospital/11798255/home?entry=bmp&from=map&fromPanelNum=2&timestamp=202309151631&x=127.1268211&y=37.4103088'],
    ["예손병원", 'https://pcmap.place.naver.com/hospital/11845111/home?entry=bmp&from=map&fromPanelNum=2&timestamp=202309151631&x=126.78745069999998&y=37.50257620182299'],
    ["부평힘찬병원", 'https://pcmap.place.naver.com/hospital/13238591/home?entry=bmp&from=map&fromPanelNum=2&timestamp=202309151632&x=126.73021550622995&y=37.49243988055428'],
    ["국제바로병원", 'https://pcmap.place.naver.com/hospital/13205824/home?entry=bmp&from=map&fromPanelNum=2&x=126.69317347360611&y=37.46551276685047&timestamp=202309151633'],
    ["좋은아침병원", 'https://pcmap.place.naver.com/hospital/36681900/home?from=map&fromPanelNum=2&x=127.131160281351&y=37.600456766549115&timestamp=202309151633'],
    ["마이크로병원", 'https://pcmap.place.naver.com/hospital/13247952/home?entry=bmp&from=map&fromPanelNum=2&x=127.45028357141803&y=36.63398620009808&timestamp=202309151634'],
    ["순천하나병원", 'https://pcmap.place.naver.com/hospital/11839329/home?entry=bmp&from=map&fromPanelNum=2&timestamp=202309151635&x=127.509923&y=34.940572'],
    ["W병원", 'https://pcmap.place.naver.com/hospital/12994580/home?entry=bmp&from=map&fromPanelNum=2&timestamp=202309151636&x=128.544905753688&y=35.8526935132234'],
    ["진주 세란병원", 'https://pcmap.place.naver.com/hospital/13261626/home?from=map&fromPanelNum=1&timestamp=202309151637&x=128.092955400002&y=35.1744786999959'],
    ["우리들병원", 'https://pcmap.place.naver.com/hospital/11698277/home?entry=bmp&from=map&fromPanelNum=2&x=127.04969449999999&y=37.5190327&timestamp=202309151638'],
    ["나누리병원 강남 본관", 'https://pcmap.place.naver.com/hospital/11535960/home?entry=bmp&from=map&fromPanelNum=2&timestamp=202309151639&x=127.034016899999&y=37.5202966999978'],
    ["서울척병원", 'https://pcmap.place.naver.com/hospital/11731301/home?entry=bmp&from=map&fromPanelNum=2&x=127.029252&y=37.6088928&timestamp=202309151641'],
    ["부산우리들병원", 'https://pcmap.place.naver.com/hospital/12075325/home?entry=bmp&from=map&fromPanelNum=2&timestamp=202309111154&x=129.0856653&y=35.2216217'],
    ["나누리수원병원", 'https://pcmap.place.naver.com/hospital/32407361/home?entry=bmp&from=map&fromPanelNum=2&x=127.04811910000201&y=37.27381440000039&timestamp=202309151643'],
    ["나누리병원 인천", 'https://pcmap.place.naver.com/hospital/37811460/home?entry=bmp&from=map&fromPanelNum=2&timestamp=202309111105&x=126.7308592&y=37.499572'],
    ["의료법인 토마스의료재단 안양윌스기념병원", 'https://pcmap.place.naver.com/hospital/21176158/home?entry=bmp&from=map&fromPanelNum=2&timestamp=202309151645&x=126.958234&y=37.3727681'],
    ["안산21세기병원", 'https://pcmap.place.naver.com/hospital/21560235/home?entry=bmp&from=map&fromPanelNum=2&timestamp=202309111125&x=126.8317939&y=37.3307826'],
    ["대전우리병원", 'https://pcmap.place.naver.com/hospital/11736944/home?entry=bmp&from=map&fromPanelNum=2&x=127.384073503173&y=36.3436560540428&timestamp=202309151646'],
    ["광주새우리병원", 'https://pcmap.place.naver.com/hospital/13264952/home?entry=bmp&from=map&fromPanelNum=2&x=126.8813934&y=35.1536997&timestamp=202309151647'],
    ["광주우리들병원", 'https://pcmap.place.naver.com/hospital/11857577/home?entry=bmp&from=map&fromPanelNum=2&timestamp=202309151648&x=126.85128717993997&y=35.14831580318819'],
    ["보광병원", 'https://pcmap.place.naver.com/hospital/13255036/home?from=map&fromPanelNum=2&timestamp=202309151649&x=128.541625000005&y=35.8367990000039'],
    ["참조은병원", 'https://pcmap.place.naver.com/hospital/13254757/home?entry=bmp&from=map&fromPanelNum=2&x=128.5516088&y=35.8303459999999&timestamp=202309151649'],
    ["우리들병원 대구", 'https://pcmap.place.naver.com/hospital/19513771/home?entry=bmp&from=map&fromPanelNum=2&x=128.6011786&y=35.8693571&timestamp=202309151650'],
    ["강북연세병원", 'https://pcmap.place.naver.com/hospital/13098049/home?entry=bmp&from=map&fromPanelNum=2&x=127.07550329951835&y=37.61918910116395&timestamp=202309151651'],
    ["뿌리병원", 'https://pcmap.place.naver.com/hospital/37632995/home?entry=bmp&from=map&fromPanelNum=2&x=127.4867179&y=36.6568617&timestamp=202309151652'],
    ["제일정형외과병원", 'https://pcmap.place.naver.com/hospital/11572484/home?entry=bmp&from=map&fromPanelNum=2&x=127.05607953039599&y=37.52282239349291&timestamp=202309151652'],
    ["서울척병원", 'https://pcmap.place.naver.com/hospital/21849601/home?entry=bmp&from=map&fromPanelNum=2&timestamp=202309151653&x=127.04868982744983&y=37.733606530826755'],
    ["센트럴병원", 'https://pcmap.place.naver.com/hospital/1826202107/home?entry=bmp&from=map&fromPanelNum=2&x=129.0090967&y=35.211259&timestamp=202309151653'],
    ["동아병원", 'https://pcmap.place.naver.com/hospital/11559222/home?entry=bmp&from=map&fromPanelNum=2&timestamp=202309151654&x=126.899950513673&y=35.1348131786764'],
    ["대구시티병원", 'https://pcmap.place.naver.com/hospital/13563115/home?entry=bmp&from=map&fromPanelNum=2&x=128.61945780764324&y=35.89350080734319&timestamp=202309151654']
]

## 테스트용
# hospital_data_test = [
#     ["안산21세기병원", 'https://pcmap.place.naver.com/hospital/21560235/home?entry=bmp&from=map&fromPanelNum=2&timestamp=202309111125&x=126.8317939&y=37.3307826'],
#     ["나누리병원 주안", 'https://pcmap.place.naver.com/hospital/37350818/home?entry=bmp&from=map&fromPanelNum=2&x=126.69068909963093&y=37.45093300081088&timestamp=202309111150']
# ]

# DataFrame 생성
df_hospital = pd.DataFrame(hospital_data, columns=["hospital", "url"])

# DataFrame 출력
# pd.set_option('display.max_colwidth', None)
# print(df_hospital)

   hospital  \
0  우리들병원 대구   
1    강북연세병원   
2      뿌리병원   
3  제일정형외과병원   
4     서울척병원   
5     센트럴병원   
6      동아병원   
7    대구시티병원   

                                                                                                                                                       url  
0                 https://pcmap.place.naver.com/hospital/19513771/home?entry=bmp&from=map&fromPanelNum=2&x=128.6011786&y=35.8693571&timestamp=202309151650  
1   https://pcmap.place.naver.com/hospital/13098049/home?entry=bmp&from=map&fromPanelNum=2&x=127.07550329951835&y=37.61918910116395&timestamp=202309151651  
2                 https://pcmap.place.naver.com/hospital/37632995/home?entry=bmp&from=map&fromPanelNum=2&x=127.4867179&y=36.6568617&timestamp=202309151652  
3   https://pcmap.place.naver.com/hospital/11572484/home?entry=bmp&from=map&fromPanelNum=2&x=127.05607953039599&y=37.52282239349291&timestamp=202309151652  
4  https://pcmap.place.naver.com/hospital/21849601/home?entry=bmp&from=map&from

### Tags
- 리뷰탭 클릭 : div.place_fixed_maintab > div > div > div > div > a:nth-child(2) > span

- 리뷰 박스 : div:nth-child(7) > div:nth-child(3) > div > div > ul > li

- 병원 이름 : #_title > span.Fc1rA

- 병원 주소 : li > div > div > div > span:nth-child(1) > span:nth-child(3)

- 작성자 : div > ul > li > div > a > div.VYGLG

- 리뷰 내용 : div.place_section_content > ul > li:nth-child({}) > div > a > span.zPfVt

- 리뷰 더보기 :  div.place_section_content > ul > li:nth-child({}) > div > a > span.rvCSr

- 리뷰 날짜 : li > div > div > div > span:nth-child(1) > span:nth-child(3)

- 방문 횟수 : li > div > div > div > span:nth-child(2)

- 더보기 클릭 : div > div > div > div.lfH3O > a


In [5]:
# 한 병원 url이 한 사이클
cnt_total_db = 0

for url in df_hospital['url']:
    # List생성, Column명 지정
    reviews_list = []
    columns_name = ['hospital', 'address', 'name', 'content', 'date', 'visits']
    
    # 네이버플레이스 사이트 접속
    browser.get(url)
    browser.implicitly_wait(3)
    hospital = browser.find_element_by_css_selector('#_title > span.Fc1rA').text # 병원명
    address = browser.find_element_by_css_selector('div > div > a > span.LDgIH').text # 주소
    print("■", hospital, address) # 병원명, 주소 출력
    
    # 리뷰탭 클릭
    browser.find_element_by_css_selector('div.place_fixed_maintab > div > div > div > div > a:nth-child(2) > span').click()
    time.sleep(2)
    # 전체 더보기 버튼 클릭
    while True:
        try:
            element = WebDriverWait(browser, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'div > div > div > div.lfH3O > a'))
            )
            element.click()
            time.sleep(1)
        except:
            try:
                element = WebDriverWait(browser, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, 'div > div > div > div.lfH3O > a'))
                )
                element.click()
                time.sleep(1)
            except:
                break

                

#    # 리뷰 더보기 버튼 개수 찾기 (화면 크기에 따라 달라짐)
#    see_more_buttons = browser.find_elements_by_css_selector('div.place_section_content > ul > li > div > a > span.rvCSr')
#    print("긴 댓글    :", len(see_more_buttons))
#    # 리뷰 더보기 클릭
#    for num_see_more in range(len(see_more_buttons)): 
#        time.sleep(1)
#        see_more_path = see_more_buttons[num_see_more]
#        see_more_path.click()
        
    # reviews_bundle : 전체 리뷰 (리뷰박스) 가져오기
    reviews_bundle = browser.find_elements_by_css_selector('div:nth-child(7) > div:nth-child(3) > div > div > ul > li')

    browser.find_element_by_css_selector('body').send_keys(Keys.HOME)
    time.sleep(1)

    for click_review, review in enumerate(reviews_bundle):
        try:
            click_review_path = browser.find_element_by_css_selector('div > ul > li:nth-child({}) > div.ZZ4OK > a > span'.format(click_review+1))
            click_review_path.click()
        except:
            try:
                time.sleep(1)
                click_review_path.click()
            except:
                print("실패")
                pass
    # 클릭한 리뷰로 다시 저장
    reviews_bundle = browser.find_elements_by_css_selector('div:nth-child(7) > div > div > div > ul > li')
    
    # 리뷰박스에서 작성자명, 리뷰내용, 날짜, 방문횟수 수집 # 리스트에 추가
    cnt = 0 # 리뷰 저장 개수 카운트
    for review in reviews_bundle:
        try :
            content = review.find_element_by_css_selector('div.place_section_content > ul > li > div.ZZ4OK > a').text
            name = review.find_element_by_css_selector('div > ul > li > div > a > div.VYGLG').text 
            date = review.find_element_by_css_selector('li > div > div > div > span:nth-child(1) > span:nth-child(3)').text
            visits = review.find_element_by_css_selector('li > div > div > div > span:nth-child(2)').text
            full_review = [hospital, address, name, content, date, visits]
            reviews_list.append(full_review)
            cnt += 1
        except : # 댓글 내용이 없고 사진만 있을 경우 제외 
            pass
        
    # reviews_list를 Mongodb에 추가     
    df_reviews = pd.DataFrame(data=reviews_list, columns=columns_name)
    data_dict = df_reviews.to_dict(orient='records')
    collection.insert_many(data_dict)        
    
    print("전체 댓글  :", len(reviews_bundle))
    print("저장된 댓글 :", cnt)  
    cnt_total_db += cnt
    print("DB 업로드 댓글 :", cnt_total_db)      

browser.quit()


■ 우리들병원 대구 대구 중구 국채보상로 648 SHL동성빌딩
전체 댓글  : 120
저장된 댓글 : 118
DB 업로드 댓글 : 118
■ 강북연세병원 서울 노원구 동일로 996
전체 댓글  : 116
저장된 댓글 : 116
DB 업로드 댓글 : 234
■ 뿌리병원 충북 청주시 청원구 내덕로 56
전체 댓글  : 1201
저장된 댓글 : 1189
DB 업로드 댓글 : 1423
■ 제일정형외과병원 서울 강남구 영동대로 726 경일빌딩
전체 댓글  : 105
저장된 댓글 : 105
DB 업로드 댓글 : 1528
■ 서울척병원 경기 의정부시 태평로 6 (경기도 의정부시 의정부동 99-2)
전체 댓글  : 95
저장된 댓글 : 93
DB 업로드 댓글 : 1621
■ 센트럴병원 부산 북구 만덕대로 37 센트럴병원
전체 댓글  : 193
저장된 댓글 : 193
DB 업로드 댓글 : 1814
■ 동아병원 광주 남구 대남대로 238
실패
전체 댓글  : 136
저장된 댓글 : 135
DB 업로드 댓글 : 1949
■ 대구시티병원 대구 북구 동북로 270
전체 댓글  : 119
저장된 댓글 : 118
DB 업로드 댓글 : 2067


In [7]:
cursor = collection.find({})
list_cursors = list(cursor)

In [8]:
df_total = pd.DataFrame(list_cursors)

In [9]:
df_total[:3]

,_id,hospital,address,name,content,date,visits
0,6503f9cf15803af914b2b3f2,서울바른병원,서울 영등포구 여의대방로 145,oyu****,"입구부터 느껴지는 청결함과, 친절한 분위기! 웃으며 인사해주신 직원분께 감사",2023년 9월 14일 목요일,1번째 방문
1,6503f9cf15803af914b2b3f3,서울바른병원,서울 영등포구 여의대방로 145,o0p****,설명 쉽게 쉽게 잘해주시고 꼼꼼함 면에서도 상당히 만족스러워요,2023년 9월 13일 수요일,1번째 방문
2,6503f9cf15803af914b2b3f4,서울바른병원,서울 영등포구 여의대방로 145,asy****,"다른데 안가고 여기서 치료 받길 잘했어요, 상담부터 편하게해주셔서 감사해요",2023년 9월 12일 화요일,1번째 방문


In [10]:
df_drop_id = df_total.drop('_id', axis=1)
df_drop_id[:3]

,hospital,address,name,content,date,visits
0,서울바른병원,서울 영등포구 여의대방로 145,oyu****,"입구부터 느껴지는 청결함과, 친절한 분위기! 웃으며 인사해주신 직원분께 감사",2023년 9월 14일 목요일,1번째 방문
1,서울바른병원,서울 영등포구 여의대방로 145,o0p****,설명 쉽게 쉽게 잘해주시고 꼼꼼함 면에서도 상당히 만족스러워요,2023년 9월 13일 수요일,1번째 방문
2,서울바른병원,서울 영등포구 여의대방로 145,asy****,"다른데 안가고 여기서 치료 받길 잘했어요, 상담부터 편하게해주셔서 감사해요",2023년 9월 12일 화요일,1번째 방문


In [12]:
df_drop_id.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15672 entries, 0 to 15671
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   hospital  15672 non-null  object
 1   address   15672 non-null  object
 2   name      15672 non-null  object
 3   content   15672 non-null  object
 4   date      15672 non-null  object
 5   visits    15672 non-null  object
dtypes: object(6)
memory usage: 734.8+ KB


In [11]:
df_drop_dup = df_drop_id.drop_duplicates()
df_drop_dup.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14102 entries, 0 to 15671
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   hospital  14102 non-null  object
 1   address   14102 non-null  object
 2   name      14102 non-null  object
 3   content   14102 non-null  object
 4   date      14102 non-null  object
 5   visits    14102 non-null  object
dtypes: object(6)
memory usage: 771.2+ KB


In [13]:
df_drop_dup.to_csv('./JungMoonChoi.csv')

In [6]:
# {"content": {"$regex": "수술"}}
# {"hospital": {"$regex": "연세백퍼센트병원"}}
# {"name": {"$regex": "동동"}}